# Setup 

In [ ]:
import os, json, time, requests 

import pandas as pd 
import numpy as np 
from unidecode import unidecode 

# folder where all the files should go 
folder_path = f"C://Users//YOUR_FILE_PATH" 

## Get League ID 

In [ ]:
# league info 
username = "YOUR_USERNAME"  
season = 2024               # change this to the season that your league is for 

# get my user id 
url = f"https://api.sleeper.app/v1/user/{username}"
user_id = requests.get(url).json()['user_id'] 

# get my leagues 
url = f"https://api.sleeper.app/v1/user/{user_id}/leagues/nba/{season}" 
leagues = requests.get(url).json() 

# get the league id 
league_id = leagues[0]['league_id'] 

# show the league id 
print(league_id)

# Users 

In [ ]:
# get the users 
url = f"https://api.sleeper.app/v1/league/{league_id}/users" 
users = requests.get(url).json() 

# loop through the users and add to a dataframe 
df = pd.DataFrame() 
for i, user in enumerate(users):
    df = pd.concat([
        df, 
        pd.DataFrame({
            "USER_ID": [user['user_id']], 
            "NAME": [user['display_name']] 
        })
    ]) 

# save to a csv 
df.to_csv(f"{folder_path}//sleeper_users.csv", index = False) 

# showcase the data 
df.head() 

# Rosters 

In [ ]:
# get the league rosters 
url = f"https://api.sleeper.app/v1/league/{league_id}/rosters" 
rosters = requests.get(url).json() 

# loop through the rosters and add the players 
df = pd.DataFrame() 
for i, roster in enumerate(rosters):

    # get the players 
    players = roster['players'] 
    starters = roster['starters'] 
    is_starter = [1 if p in starters else 0 for p in players] 

    # add to the dataframe 
    df = pd.concat([
        df, 
        pd.DataFrame({
            "ROSTER_ID": roster.get("roster_id", None),
            "OWNER_ID": roster.get("owner_id", None), 
            "PLAYER_ID_SLEEPER": roster["players"], 
            "IS_STARTER": is_starter 
        })
    ], ignore_index = True) 

# save to a csv 
df.to_csv(f"{folder_path}//sleeper_rosters.csv", index = False) 

# showcase the data 
df.head() 

# Players 

In [ ]:
# call the API to get the players 
url = f"https://api.sleeper.app/v1/players/nba" 
response = requests.get(url)  
players = response.json() 

# function to convert to integer 
def convert_int(x): 
    try: 
        return int(x) 
    except: 
        return -1 

# loop through the players and add to the dataframe 
df = pd.DataFrame() 
for k, val in players.items(): 

    # get the player positions 
    positions = val.get('fantasy_positions', []) 
    if positions is None: 
        positions = [] 
    
    df = pd.concat([df, pd.DataFrame({
        "PLAYER_ID_SLEEPER": [convert_int(k)],  
        "FIRST_NAME": val.get('first_name', None), 
        "LAST_NAME": val.get('last_name', None), 
        "TEAM_ABBR": val.get('team', None), 
        "POSITIONS": ", ".join(positions), 
        "TEAM_STATUS": val.get('status', None), 
        "SEARCH_RANK": val.get('search_rank', None), 
        "DC_POSITION": val.get('depth_chart_position', None) 
    })], ignore_index = True) 

# remove accents 
df["FIRST_NAME"] = df["FIRST_NAME"].apply(lambda x: unidecode(str(x))) 
df["LAST_NAME"] = df["LAST_NAME"].apply(lambda x: unidecode(str(x))) 

# concatenate player names 
df["PLAYER_NAME"] = df["FIRST_NAME"] + " " + df["LAST_NAME"] 

# showcase the total number of players 
nplayers = df.shape[0] 
print(f"Total players: {nplayers:,}") 

# save to a csv 
df.to_csv(f"{folder_path}//sleeper_players.csv", index = False)

# showcase the data 
df.head() 